In [1]:
import torch
import torchvision
import os
import pandas as pd
import numpy as np
from skimage import io
import matplotlib.pyplot as plt
import tqdm
from torch import nn
import torch.nn.functional as F
from skimage import transform
from torch.autograd import Variable
import logging

In [2]:
with open('../eval/list_attr_celeba.txt') as file:
    n = int(file.readline())
    columns = file.readline().strip().split()
    labels = []
    fnames = []
    for line in file:
        fields = line.strip().split()
        fnames.append(fields[0])
        labels.append(np.array(list(map(int, fields[1:]))))
    labels = np.stack(labels)
    fnames = np.array(fnames)
f_ids = dict(zip(fnames, range(len(fnames))))
labels = (labels + 1) // 2

In [3]:
def load_ims(fnames, addr='../img_align_celeba/'):
    ims = []
    for fname in fnames:
        ims.append(io.imread(addr + fname))
    return np.stack(ims)

In [4]:
def ran_transform(X):
    height = X.shape[1]# // 2
    width = X.shape[2]# // 2
    X_out = np.zeros((X.shape[0], height, width, 3), dtype=np.float32)
    
    for idx, x in enumerate(X):
        x = transform.resize(x, (height, width))
        angle = np.random.uniform(-10, 10)
        h_scale = np.random.randint(0, 10)
        v_scale = np.random.randint(0, 10)
        h_pos = np.random.randint(0, h_scale + 1)
        v_pos = np.random.randint(0, v_scale + 1)
        x = transform.rotate(x, angle)
        x = transform.resize(x, (height + v_scale, width + h_scale))
        x = x[v_pos:height + v_pos, h_pos: width + h_pos]
        X_out[idx] = x
    X_out = X_out.transpose([0, 3, 1, 2])
    return X_out

def test_transform(X):
    height = X.shape[1] #// 2
    width = X.shape[2] #// 2
    X_out = np.zeros((X.shape[0], height, width, 3), dtype=np.float32)
    for idx, x in enumerate(X):
        x = transform.resize(x, (height, width))
        X_out[idx] = x
    X_out = X_out.transpose([0, 3, 1, 2])
    return X_out

In [5]:
def batch_gen(fnames, labels, batch_size, epoch_size, addr='../img_align_celeba/'):
    for bn in range(epoch_size):
        ids = np.random.randint(0, len(fnames), batch_size)
        X = ran_transform(load_ims(fnames[ids], addr))
        y = labels[ids]
        yield X, y

In [6]:
class MyVGG(nn.Module):
    
    def __init__(self):
        super(MyVGG, self).__init__()
        self.features = torchvision.models.squeezenet1_0(pretrained=False).features
        #for par in self.features.parameters():
        #    par.requires_grad=False
        #15360
        #66560
        self.embeddings = nn.Sequential(nn.Linear(66560, 1024), nn.ReLU(inplace=True), nn.Dropout(0.5),
                                        #nn.Linear(2048, 1024), nn.ReLU(inplace=True), nn.Dropout(0.5),
                                        nn.Linear(1024, 128))
        self.classif = nn.Sequential(self.embeddings, nn.ReLU(), nn.Linear(128, 40))
        
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.shape[0], -1)
        # print(x.shape)
        return F.sigmoid(self.classif(x))
    
    def get_embeddings(self, x):
        x = self.features(x)
        x = x.view(x.shape[0], -1)
        x = F.relu(self.embeddings(x))
 
        return F.normalize(self.embeddings_layer(x), dim=1)

In [7]:
class Logger:
    def __init__(self, filename):
        self.filename = filename
        self.logstack = []
        
    def log(self, text):
        self.logstack.append(text)
        
    def flush(self):

        with open(self.filename, 'a') as f:
            for text in self.logstack:
                f.write(text + '\n')
        self.logstack = []
        

In [8]:
from sklearn.model_selection import train_test_split
train_names, test_names = train_test_split(fnames, test_size=0.03, random_state=1234)
X_test = test_transform(load_ims(test_names))
y_test = labels[np.array([f_ids[name] for name in test_names])]
y_train = labels[np.array([f_ids[name] for name in train_names])]

/usr/local/lib/python3.5/dist-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [9]:
torch.cuda.set_device(5)

In [10]:
net = MyVGG().cuda()
optimizer = torch.optim.Adam(net.parameters(), lr=0.0001)
criterion = F.binary_cross_entropy

In [10]:
epoch = 0

In [11]:
net = MyVGG()
net.load_state_dict(torch.load('model_classif'))
net.cuda()
optimizer = torch.optim.Adam(net.parameters(), lr=0.00001)
criterion = F.binary_cross_entropy

In [ ]:
batch_size = 50
epoch_size = 50
n_epochs = 3000
logger = Logger('logs3.txt')
curloss = 100000
for n in range(n_epochs):
    net.train()
    for b_id, (x, y) in enumerate(batch_gen(train_names, y_train, batch_size, epoch_size)):
        x = Variable(torch.FloatTensor(x).cuda())
        y = Variable(torch.FloatTensor(y).cuda())
        optimizer.zero_grad()
        out = net(x)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()
        lossv = loss.data.cpu().numpy()[0]
        logger.log(' '.join(['train', str(epoch), str(b_id), str(lossv)]))
    net.eval()
    test_ids = np.random.randint(0, X_test.shape[0], batch_size)
    x = Variable(torch.FloatTensor(X_test[test_ids]).cuda())
    y = Variable(torch.FloatTensor(y_test[test_ids]).cuda())
    out = net(x)
    loss = criterion(out, y)
    lossv = loss.data.cpu().numpy()[0]
    logger.log(' '.join(['test', str(epoch), str(lossv)]))
    if lossv < curloss:
        curloss = lossv
        torch.save(net.state_dict(), 'model')
    epoch += 1
    logger.flush()
        

/usr/local/lib/python3.5/dist-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [11]:
net.cpu()

MyVGG(
  (features): Sequential(
    (0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), dilation=(1, 1), ceil_mode=True)
    (3): Fire(
      (squeeze): Conv2d(96, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace)
    )
    (4): Fire(
      (squeeze): Conv2d(128, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace)
    )
    (5): Fire(
      (squeeze): Conv2d(128, 32, kernel

In [12]:
torch.save(net.state_dict(), 'model_classif')